In [3]:
from __future__ import print_function
import torch
import pandas as pd

In [2]:
x = torch.Tensor(5,3)
print(x)


 0.0000e+00 -0.0000e+00  0.0000e+00
-0.0000e+00  4.3074e-08  2.5202e-09
 2.6660e-09  4.2468e-08  5.1888e-11
 7.1450e+31  2.1658e+23  4.1731e+21
 0.0000e+00 -0.0000e+00 -2.2052e+23
[torch.FloatTensor of size 5x3]



In [5]:
x = torch.rand(5,3)
print(x)


 0.8800  0.1544  0.7321
 0.6255  0.2352  0.9704
 0.4951  0.5528  0.1874
 0.0786  0.4899  0.7012
 0.5576  0.3424  0.1393
[torch.FloatTensor of size 5x3]



In [7]:
y = torch.rand(5,3)
y


 0.6071  0.4543  0.4356
 0.9595  0.1023  0.2235
 0.1310  0.4608  0.5363
 0.2058  0.8654  0.7069
 0.8077  0.2315  0.5427
[torch.FloatTensor of size 5x3]

In [8]:
result = torch.Tensor(5,3)
torch.add(x,y, out=result)
result


 1.4871  0.6087  1.1678
 1.5850  0.3375  1.1940
 0.6261  1.0136  0.7237
 0.2844  1.3553  1.4080
 1.3654  0.5739  0.6820
[torch.FloatTensor of size 5x3]

In [9]:
y.add_(x)
y


 1.4871  0.6087  1.1678
 1.5850  0.3375  1.1940
 0.6261  1.0136  0.7237
 0.2844  1.3553  1.4080
 1.3654  0.5739  0.6820
[torch.FloatTensor of size 5x3]

In [12]:
print(x[:,2])


 0.7321
 0.9704
 0.1874
 0.7012
 0.1393
[torch.FloatTensor of size 5]



In [15]:
a = torch.ones(5)
b = a.numpy()  # numpy representation of a. b points to a eventually
b

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)

In [17]:
a.add_(1)
print(a)
print(b)


 3
 3
 3
 3
 3
[torch.FloatTensor of size 5]

[ 3.  3.  3.  3.  3.]


# Autograd Tutorial 

In [18]:
import torch
from torch.autograd import Variable

In [19]:
x = Variable(torch.ones(2,2), requires_grad=True)
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



In [20]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



In [21]:
print(y.grad_fn)

In [22]:
z = y*y*3
out = z.mean()

print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



In [23]:
out.backward()

In [24]:
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



In [26]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y*2
print(y)

Variable containing:
 0.8486
-1.0598
-0.2457
[torch.FloatTensor of size 3]

Variable containing:
  869.0136
-1085.2010
 -251.5711
[torch.FloatTensor of size 3]



In [28]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
  409.6000
 4096.0000
    0.4096
[torch.FloatTensor of size 3]



# Neural Networks

In [4]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [5]:
net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [8]:
params = list(net.parameters())
print(len(params))
for i in range(10):
    print(params[i].size())
# print(params[1].size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [9]:
input = Variable(torch.randn(1,1,32,32))
out = net(input)
print(out)

Variable containing:
-0.1230  0.0296 -0.0383  0.0359 -0.0231 -0.0890 -0.0209  0.1452  0.1538  0.0645
[torch.FloatTensor of size 1x10]



In [10]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [11]:
output = net(input)
target = Variable(torch.arange(1, 11))  # a dummy target, for example
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

Variable containing:
 38.0356
[torch.FloatTensor of size 1]



In [16]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [17]:
net.zero_grad()     # zeroes the gradient buffers of all parameters
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
# Starting backprop
loss.backward()
# End backprop
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
 -0.1511
 -2.8405
  0.6803
  3.9805
 -5.9962
 -9.0394
[torch.FloatTensor of size 6]



In [18]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [19]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update